In [12]:
%matplotlib inline
import pandas as pd
import numpy as np
from numpy import arange
import math

import seaborn as sns
sns.set_style("white")

import scipy as sp
from scipy import linalg, optimize

import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

import collections
from collections import Counter

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.path as mpath
from matplotlib.dates import DateFormatter

import calendar
import datetime as dt
fromtimestamp = dt.datetime.fromtimestamp


In [13]:
# import raw data and confirm
df = pd.read_csv('../Data/Raw/ufo_raw.csv')
df.head(1)

,datetime,City,State,Shape,Duration,Summary,Posted,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,2006-01-01 01:20:00,I-80 (unknown city proximity),NY,Formation,0 days 00:00:20.000000000,"Traveling from Brooklyn, NY to Groton, NY on I...",6/7/19,NaN,NaN,NaN,NaN


In [14]:
# copy intow(orking)df
wdf = df.copy()
# review columns
wdf.columns

Index(['datetime', 'City', 'State', 'Shape', 'Duration', 'Summary', 'Posted',
       'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10'],
      dtype='object')

In [15]:
# add col datetype
wdf['datetype'] = pd.to_datetime(wdf['datetime']).dt.date

In [16]:
# add col dtdtype
wdf['dttyp'] = pd.to_datetime(wdf['datetype'])

In [17]:
# add col weekofyear
wdf['weekofyear'] = wdf['dttyp'].dt.isocalendar().week

In [18]:
# add col year
wdf['year'] = wdf['dttyp'].dt.year

In [19]:
# add col year_week
wdf['year_week'] = wdf['year'].astype(str) + "-" + wdf['weekofyear'].astype(str)

In [20]:
# drop useless cols and update wdf
wdf = wdf.drop(columns=['Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'datetype','datetime'])

In [21]:
# new df "ufodata"
ufodata = pd.DataFrame(wdf['year_week'])

In [22]:
# add col date_of_sight
ufodata['date_of_sight'] = wdf['dttyp']

In [23]:
# add cols for year, month, week_of_year,state,city,sight_summary, shape
ufodata['year'] = wdf['year']
ufodata['month'] = wdf['dttyp'].dt.month
ufodata['week_of_year'] = wdf['weekofyear']
ufodata['state'] = wdf['State']
ufodata['city'] = wdf['City']
ufodata['sight_summary'] = wdf['Summary']
ufodata['ufo_shape'] = wdf['Shape']

# confirm
ufodata.head()

,year_week,date_of_sight,year,month,week_of_year,state,city,sight_summary,ufo_shape
0,2006-52,2006-01-01,2006,1,52,NY,I-80 (unknown city proximity),"Traveling from Brooklyn, NY to Groton, NY on I...",Formation
1,2006-52,2006-01-01,2006,1,52,TX,San Antonio/Laredo (between),Three separate sightins which ocurred in rural...,Triangle
2,2006-1,2006-01-02,2006,1,1,MI,Ann Arbor,"Probable HOAX: 5 ufos, silent black objects, m...",Changing
3,2006-1,2006-01-02,2006,1,1,VA,Fredericksburg,clear sky looked like a star 10:00 am in the m...,Light
4,2006-1,2006-01-04,2006,1,1,PA,Scotland,"Small, white, light in the northeast sky...dis...",Light


In [160]:
# make new df "datedf" with daterange

dt = pd.date_range("20060102","20200906", freq='W')
datedf = pd.DataFrame(dt).rename(columns={0:'startdayweek'})

# add cols weeknum,month,year,week_id
datedf['weeknum'] = datedf['startdayweek'].dt.isocalendar().week
datedf['month'] = datedf['startdayweek'].dt.month
datedf['year'] = datedf['startdayweek'].dt.year
datedf['week_id'] = datedf['year'].astype(str)+"-"+datedf['weeknum'].astype(str)

In [161]:
# make new df ufocount from ufodata.copy, drop cols we don't need
ufocount = ufodata.copy().drop(columns=['index','state','city','sight_summary','ufo_shape']).rename(columns={'year_week':'week_id'} ).sort_values('week_id').reset_index()

,index,week_id,date_of_sight,year,month,week_of_year
0,2,2006-1,2006-01-02,2006,1,1


In [163]:
# merge date df and count df to ufo_cleaned_fin
ufo_cleaned_fin = pd.merge(left=datedf, right=ufocount, left_on='week_id', right_on='week_id')

In [164]:
# review
ufo_cleaned_fin.head()

,startdayweek,weeknum,month_x,year_x,week_id,index,date_of_sight,year_y,month_y,week_of_year
0,2006-01-08,1,1,2006,2006-1,2,2006-01-02,2006,1,1
1,2006-01-08,1,1,2006,2006-1,3,2006-01-02,2006,1,1
2,2006-01-08,1,1,2006,2006-1,4,2006-01-04,2006,1,1
3,2006-01-15,2,1,2006,2006-2,5,2006-01-12,2006,1,2
4,2006-01-15,2,1,2006,2006-2,6,2006-01-12,2006,1,2
...,...,...,...,...,...,...,...,...,...,...
71616,2020-09-06,36,9,2020,2020-36,70922,2020-08-31,2020,8,36
71617,2020-09-06,36,9,2020,2020-36,70923,2020-08-31,2020,8,36
71618,2020-09-06,36,9,2020,2020-36,70920,2020-08-31,2020,8,36
71619,2020-09-06,36,9,2020,2020-36,70925,2020-08-31,2020,8,36


In [165]:
# copy to wdfweek df and drop cols we do not need
wdfweek = ufo_cleaned_fin.copy().drop(columns=['weeknum','index','year_x','month_x'])

,startdayweek,week_id,date_of_sight,year_y,month_y,week_of_year
0,2006-01-08,2006-1,2006-01-02,2006,1,1
1,2006-01-08,2006-1,2006-01-02,2006,1,1
2,2006-01-08,2006-1,2006-01-04,2006,1,1
3,2006-01-15,2006-2,2006-01-12,2006,1,2
4,2006-01-15,2006-2,2006-01-12,2006,1,2


In [166]:
# store counts per week_id in "ufosperweek" df
ufosperweek = pd.DataFrame(wdfweek['startdayweek'].value_counts()).reset_index().rename(columns={'index':'startdateweek','startdayweek': 'weekly_ufocount'})
# review
ufosperweek.head(1)

,startdateweek,weekly_ufocount
0,2014-07-06,470


In [167]:
# set max count to 100 percent
maxalltime = ufosperweek.weekly_ufocount.max() 

# store val for one percent
onepct = maxalltime/100

In [168]:
# new col with relative percentage vals
ufosperweek['rel_val_alltime'] = ufosperweek['weekly_ufocount']/onepct

In [169]:
# add all cols we want
ufosperweek['year'] = ufosperweek['startdateweek'].dt.year
ufosperweek['month'] = ufosperweek['startdateweek'].dt.month
ufosperweek['weeknum'] = ufosperweek['startdateweek'].dt.isocalendar().week 
ufosperweek['week_id'] = ufosperweek['year'].astype(str) + "-" + ufosperweek['weeknum'].astype(str)

In [170]:
# review
ufosperweek.head()

,startdateweek,weekly_ufocount,rel_val_alltime,year,month,weeknum,week_id
0,2014-07-06,470,100.000000,2014,7,27,2014-27
1,2015-11-08,406,86.382979,2015,11,45,2015-45
2,2020-04-19,403,85.744681,2020,4,16,2020-16
3,2013-07-07,399,84.893617,2013,7,27,2013-27
4,2012-07-08,339,72.127660,2012,7,27,2012-27


In [24]:
# export csv and set to comment after
#ufosperweek.to_csv('../Data/Datasets/cleandate_weekly_reval.csv')